Mount Google Drive Account

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
from pathlib import Path
import os
import cv2

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Install tensorflow

In [ ]:
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt

from numpy.random import seed
seed(1)
tf.random.set_seed(2)

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices())
print(tf.config.list_physical_devices('GPU'))

2.4.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Point to training data

In [ ]:
base_dir = '/content/gdrive/MyDrive/FYP/Consistent_dataset/Augmented_train_images'

Generate training and validation data

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 32

# datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=1./255, 
#     validation_split=0.2)
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0)



train_generator = datagen.flow_from_directory(
    "/content/gdrive/MyDrive/FYP/Consistent_dataset/Augmented_train_images",
    seed=123,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

# train_generator = datagen.flow_from_directory(
#     base_dir,
#     seed=123,
#     target_size=(IMAGE_SIZE, IMAGE_SIZE),
#     batch_size=BATCH_SIZE, 
#     subset='training')


val_generator = datagen.flow_from_directory(
    "/content/gdrive/MyDrive/FYP/Consistent_dataset/validation_full_images",
    seed=123,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')


# val_generator = datagen.flow_from_directory(
#     base_dir,
#     seed=123,
#     target_size=(IMAGE_SIZE, IMAGE_SIZE),
#     batch_size=BATCH_SIZE, 
#     subset='validation')

Found 3036 images belonging to 6 classes.
Found 22 images belonging to 6 classes.


In [ ]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((32, 224, 224, 3), (32, 6))

Save the labels in a file which will be downloaded later.

In [ ]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

number_of_classes = len(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'Atlantic_Puffin': 0, 'Eurasian_Bittern': 1, 'European_Nightjar': 2, 'Northern_Raven': 3, 'Red-throated_Loon': 4, 'Tui': 5}


In [ ]:
!cat labels.txt

Atlantic_Puffin
Eurasian_Bittern
European_Nightjar
Northern_Raven
Red-throated_Loon
Tui

###  Create the model

In [ ]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.VGG19(input_shape=IMG_SHAPE, include_top=False)

In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(number_of_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 32)          147488    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 20,172,070
Trainable params: 147,686
Non-trainable params: 20,024,384
_________________________________________________________________


In [ ]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 4


Train the initial model

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from sys import maxsize

epochs = 20

history = model.fit(train_generator, 
                    steps_per_epoch=len(train_generator), 
                    epochs=epochs, 
                    validation_data=val_generator, 
                    validation_steps=len(val_generator))

Epoch 1/20
95/95 [==============================] - 1642s 17s/step - loss: 1.7028 - accuracy: 0.3076 - val_loss: 1.5367 - val_accuracy: 0.4091
Epoch 2/20
95/95 [==============================] - 50s 523ms/step - loss: 1.3839 - accuracy: 0.4979 - val_loss: 1.3436 - val_accuracy: 0.5909
Epoch 3/20
95/95 [==============================] - 50s 525ms/step - loss: 1.1760 - accuracy: 0.5946 - val_loss: 1.2332 - val_accuracy: 0.5455
Epoch 4/20
95/95 [==============================] - 50s 528ms/step - loss: 1.0255 - accuracy: 0.6474 - val_loss: 1.1143 - val_accuracy: 0.6364
Epoch 5/20
95/95 [==============================] - 50s 525ms/step - loss: 0.9143 - accuracy: 0.6904 - val_loss: 1.1652 - val_accuracy: 0.5909
Epoch 6/20
95/95 [==============================] - 50s 526ms/step - loss: 0.8302 - accuracy: 0.7210 - val_loss: 1.1009 - val_accuracy: 0.6364
Epoch 7/20
95/95 [==============================] - 50s 527ms/step - loss: 0.7505 - accuracy: 0.7587 - val_loss: 1.0514 - val_accuracy: 0.5000

Fine tuning

In [ ]:
base_model.trainable = True

print("Number of layers in the base model: ", len(base_model.layers))

model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

model.summary()
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of layers in the base model:  22
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 32)          147488    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 20,172,070
Trainable params: 20,172,070
Non-trainable params: 0
_________________________________________________________________
Number of trainable variables = 36


In [ ]:
epochs = 30
early_stop = EarlyStopping(monitor="val_accuracy", mode="max", verbose=1, min_delta=0.01, patience=epochs, restore_best_weights=True)

history_fine = model.fit(train_generator, 
                         steps_per_epoch=len(train_generator), 
                         epochs=epochs, 
                         validation_data=val_generator, 
                         validation_steps=len(val_generator), callbacks=[early_stop])

Epoch 1/30
95/95 [==============================] - 52s 536ms/step - loss: 0.3965 - accuracy: 0.8660 - val_loss: 0.8952 - val_accuracy: 0.6364
Epoch 2/30
95/95 [==============================] - 51s 539ms/step - loss: 0.1610 - accuracy: 0.9469 - val_loss: 0.9025 - val_accuracy: 0.7273
Epoch 3/30
95/95 [==============================] - 51s 540ms/step - loss: 0.1043 - accuracy: 0.9660 - val_loss: 0.8846 - val_accuracy: 0.7727
Epoch 4/30
95/95 [==============================] - 51s 537ms/step - loss: 0.0587 - accuracy: 0.9804 - val_loss: 0.7442 - val_accuracy: 0.7273
Epoch 5/30
95/95 [==============================] - 51s 532ms/step - loss: 0.0603 - accuracy: 0.9799 - val_loss: 1.0557 - val_accuracy: 0.7273
Epoch 6/30
95/95 [==============================] - 51s 534ms/step - loss: 0.0495 - accuracy: 0.9824 - val_loss: 1.9527 - val_accuracy: 0.5455
Epoch 7/30
95/95 [==============================] - 51s 531ms/step - loss: 0.0419 - accuracy: 0.9866 - val_loss: 0.8253 - val_accuracy: 0.7273

Download model and labels

In [ ]:
saved_model_dir = 'save/fine_tuning'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('/content/gdrive/MyDrive/FYP/model.tflite', 'wb') as f:
  f.write(tflite_model)

from google.colab import files

files.download('/content/gdrive/MyDrive/FYP/model.tflite')
files.download('labels.txt')

INFO:tensorflow:Assets written to: save/fine_tuning/assets


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>